In [ ]:
import asyncio
import json
from typing import Any, Dict, Optional

from fastmcp.client import Client
from fastmcp.client.transports import StreamableHttpTransport


In [ ]:
# ----------------------------
# CONFIG — set your server URL
# ----------------------------
server_url = "http://188.34.76.92:8002/intervista"
SESSION_HEADER = "my-mantix-backend-session"
OUTPUT_PATH = "all_product_schemas.json"
# ----------------------------
# Products list with IDs and names
# ----------------------------
products_json_str = """
[
  {
    "id": 64,
    "name": "FirmenSchutzbrief by ELEMENT"
  },
  {
    "id": 3415,
    "name": "HandySchutzbrief Basis ohne Diebstahl (Risikoträger ELEMENT)"
  },
  {
    "id": 3424,
    "name": "HandySchutzbrief Basis ohne Diebstahl (Risikoträger ELEMENT)"
  },
  {
    "id": 3436,
    "name": "HandySchutzbrief Premium mit Diebstahl (Risikoträger ELEMENT)"
  },
  {
    "id": 3448,
    "name": "HandySchutzbrief Premium Online mit Diebstahl (Risikoträger ELEMENT)"
  },
  {
    "id": 3571,
    "name": "HandySchutzbrief Mobilfunk Basis ohne Diebstahl (Risikoträger ELEMENT)"
  },
  {
    "id": 3610,
    "name": "HandySchutzbrief Mobilfunk Premium ohne Diebstahl (Risikoträger HELVETIA)"
  }
]
"""

products_list = json.loads(products_json_str)


In [ ]:
async def fetch_schema(client: Client, pid: int) -> Optional[Any]:
    params = {"id": str(pid)}
    result = await client.call_tool_mcp("get_form", params)
    if not getattr(result, "content", None):
        return None

    text = getattr(result.content[0], "text", None)
    if text is None:
        return None

    # Try parsing JSON, otherwise keep raw string
    try:
        return json.loads(text)
    except Exception:
        return text


async def gather_schemas():
    transport = StreamableHttpTransport(
        url=server_url,
        headers={"X-Mantix-Session": SESSION_HEADER},
    )

    output = []

    async with Client(transport) as client:
        for product in products_list:
            pid = product["id"]
            name = product["name"]
            schema = await fetch_schema(client, pid)
            output.append({
                "id": pid,
                "name": name,
                "schema": schema,
            })

    with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
        json.dump(output, f, ensure_ascii=False, indent=2)

    print(f"Wrote {OUTPUT_PATH} with {len(output)} products.")
    return output

In [ ]:
schemas = await gather_schemas()
schemas  # This will display the list in notebook